# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [38]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [12]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['example_playlist']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [13]:
print(playlist_songs[0])

{'_id': ObjectId('5e83559b93d27f4463566fd0'), 'track': 'Hit It', 'track_id': '7JXdE4b0hpNSlZmPyopgii', 'artist_name': ['American Authors'], 'artist_id': ['0MlOPi3zIDMVrfA9R04Fe3'], 'genres': ['indie poptimism', 'modern alternative rock', 'modern rock', 'neo mellow', 'pop rock'], 'danceability': 0.645, 'energy': 0.919, 'key': 7, 'loudness': -3.669, 'mode': 1, 'speechiness': 0.0853, 'acousticness': 0.0025, 'instrumentalness': 0, 'liveness': 0.527, 'valence': 0.77, 'tempo': 120.057, 'duration_ms': 207453, 'time_signature': 4}


**Next we need to make a list of every song's genres**

In [15]:
genre_list = []
for song in playlist_songs:
    genre_list.extend(song['genres'])
genre_list

['alternative dance',
 'alternative rock',
 'dance-punk',
 'garage rock',
 'indie pop',
 'indie rock',
 'indietronica',
 'modern rock',
 'new rave',
 'rock',
 'electropop',
 'indie pop',
 'indie poptimism',
 'indietronica',
 'metropopolis',
 'neo-synthpop',
 'shimmer pop',
 'alternative dance',
 'indietronica',
 'new rave',
 'shimmer pop',
 'kiwi rock',
 'nz hip hop',
 'polynesian pop',
 'wrestling',
 'dutch indie',
 'dutch indie rock',
 'dutch rock',
 'house',
 'pop',
 'tropical house',
 'uk dance',
 'alternative dance',
 'australian dance',
 'australian electropop',
 'dance-punk',
 'electropop',
 'indie pop',
 'indietronica',
 'neo-synthpop',
 'new rave',
 'pop',
 'garage rock',
 'indie rock',
 'indietronica',
 'modern rock',
 'new rave',
 'oxford indie',
 'rock',
 'argentine indie',
 'argentine rock',
 'latin alternative',
 'latin funk',
 'latin rock',
 'rock en espanol',
 'rock nacional',
 'pop',
 'tropical house',
 'afrofuturismo brasileiro',
 'baile pop',
 'brazilian hip hop',
 '

**Next we need to find the most common occuring genres to base our recommendations on**

In [27]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['modern rock', 'indie pop', 'indietronica', 'alternative dance', 'indie rock', 'new rave', 'rock', 'shimmer pop']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [17]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['modern%20rock', 'indie%20pop', 'indietronica', 'alternative%20dance', 'indie%20rock']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [24]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [28]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
print(matches)

['alternative', 'dance', 'indie', 'pop', 'rock']


In [29]:
query_string = ','.join(matches)
print(query_string)

alternative,dance,indie,pop,rock


In [34]:
recommended_tracks = sp.recommendations(seed_genres = matches, limit=20)
print(recommended_tracks['tracks'])

[{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6MDME20pz9RveH9rEXvrOM'}, 'href': 'https://api.spotify.com/v1/artists/6MDME20pz9RveH9rEXvrOM', 'id': '6MDME20pz9RveH9rEXvrOM', 'name': 'Clean Bandit', 'type': 'artist', 'uri': 'spotify:artist:6MDME20pz9RveH9rEXvrOM'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/5J1vKkCC6aoOGZjE7YZrHE'}, 'href': 'https://api.spotify.com/v1/albums/5J1vKkCC6aoOGZjE7YZrHE', 'id': '5J1vKkCC6aoO

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [37]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [72]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Tears (feat. Louisa Johnson),7py16W5fWYLFFS6BElKAjn,0.605,0.770,5,-3.645,0,0.0446,0.043100,0.000000,0.1590,0.2980,130.037,225915,4
1,Girls,5WSdMcWTKRdN1QYVJHJWxz,0.657,0.913,11,-4.984,1,0.0528,0.003920,0.001490,0.4660,0.9060,108.036,254620,4
2,Race for the Prize - 2017 Remaster,1RZlGlz8eXT1KmJYpqFRzH,0.326,0.841,0,-7.140,1,0.0858,0.000046,0.028000,0.2510,0.4490,130.723,258912,4
3,Work,32lmL4vQAAotg6MrJnhlQZ,0.725,0.534,11,-6.238,1,0.0946,0.075200,0.000000,0.0919,0.5580,91.974,219320,4
4,Master Of None,2zkTK6l0Du0nEgwPHGGT0p,0.497,0.347,5,-15.256,1,0.0260,0.620000,0.529000,0.0942,0.2120,86.486,199413,4
5,Baby Blue,1jhtxc7ON8ZzgvWGPwWXUN,0.468,0.331,3,-11.052,1,0.0360,0.613000,0.001080,0.1290,0.0900,102.448,215507,4
6,One of Us,3zSaWBoojpQGDPO2oIL80H,0.503,0.908,4,-3.681,1,0.0598,0.216000,0.000000,0.3660,0.8570,200.147,260093,4
7,Finally - Radio Edit,3YtL6y0eib6vgPmvF3mUed,0.709,0.770,11,-5.883,0,0.0446,0.016600,0.000000,0.3350,0.7720,125.027,199488,4
8,Do I Wanna Know?,3jfr0TF6DQcOLat8gGn7E2,0.548,0.532,5,-7.596,1,0.0323,0.186000,0.000263,0.2170,0.4050,85.030,272394,4
9,Unbreakable,15TC83LO1QAu0BZLRdHw01,0.457,0.948,0,-4.318,1,0.0555,0.000022,0.002250,0.1520,0.1570,139.022,249507,4


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [73]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [74]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [75]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [76]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [77]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [78]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [80]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [86]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,Tears (feat. Louisa Johnson),7py16W5fWYLFFS6BElKAjn,1
1,Girls,5WSdMcWTKRdN1QYVJHJWxz,1
2,Race for the Prize - 2017 Remaster,1RZlGlz8eXT1KmJYpqFRzH,1
3,Work,32lmL4vQAAotg6MrJnhlQZ,1
4,Master Of None,2zkTK6l0Du0nEgwPHGGT0p,0
5,Baby Blue,1jhtxc7ON8ZzgvWGPwWXUN,0
6,One of Us,3zSaWBoojpQGDPO2oIL80H,1
7,Finally - Radio Edit,3YtL6y0eib6vgPmvF3mUed,1
8,Do I Wanna Know?,3jfr0TF6DQcOLat8gGn7E2,1
9,Unbreakable,15TC83LO1QAu0BZLRdHw01,1


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [90]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'Tears (feat. Louisa Johnson)', 'Track_id': '7py16W5fWYLFFS6BElKAjn', 'Prediction': 1}
{'Track Name': 'Girls', 'Track_id': '5WSdMcWTKRdN1QYVJHJWxz', 'Prediction': 1}
{'Track Name': 'Race for the Prize - 2017 Remaster', 'Track_id': '1RZlGlz8eXT1KmJYpqFRzH', 'Prediction': 1}
{'Track Name': 'Work', 'Track_id': '32lmL4vQAAotg6MrJnhlQZ', 'Prediction': 1}
{'Track Name': 'Master Of None', 'Track_id': '2zkTK6l0Du0nEgwPHGGT0p', 'Prediction': 0}
{'Track Name': 'Baby Blue', 'Track_id': '1jhtxc7ON8ZzgvWGPwWXUN', 'Prediction': 0}
{'Track Name': 'One of Us', 'Track_id': '3zSaWBoojpQGDPO2oIL80H', 'Prediction': 1}
{'Track Name': 'Finally - Radio Edit', 'Track_id': '3YtL6y0eib6vgPmvF3mUed', 'Prediction': 1}
{'Track Name': 'Do I Wanna Know?', 'Track_id': '3jfr0TF6DQcOLat8gGn7E2', 'Prediction': 1}
{'Track Name': 'Unbreakable', 'Track_id': '15TC83LO1QAu0BZLRdHw01', 'Prediction': 1}
{'Track Name': 'Champagne Supernova', 'Track_id': '4Jgp57InfWE4MxJLfheNVz', 'Prediction': 0}
{'Track Name': 'S

In [94]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': 'Tears (feat. Louisa Johnson)', 'Track_id': '7py16W5fWYLFFS6BElKAjn', 'Prediction': 1}
{'Track Name': 'Girls', 'Track_id': '5WSdMcWTKRdN1QYVJHJWxz', 'Prediction': 1}
{'Track Name': 'Race for the Prize - 2017 Remaster', 'Track_id': '1RZlGlz8eXT1KmJYpqFRzH', 'Prediction': 1}
{'Track Name': 'Work', 'Track_id': '32lmL4vQAAotg6MrJnhlQZ', 'Prediction': 1}
{'Track Name': 'One of Us', 'Track_id': '3zSaWBoojpQGDPO2oIL80H', 'Prediction': 1}
{'Track Name': 'Finally - Radio Edit', 'Track_id': '3YtL6y0eib6vgPmvF3mUed', 'Prediction': 1}
{'Track Name': 'Do I Wanna Know?', 'Track_id': '3jfr0TF6DQcOLat8gGn7E2', 'Prediction': 1}
{'Track Name': 'Unbreakable', 'Track_id': '15TC83LO1QAu0BZLRdHw01', 'Prediction': 1}
{'Track Name': 'Superheroes', 'Track_id': '186hvCTyrni4KT9nwIQ7zS', 'Prediction': 1}
{'Track Name': 'Wanna Sip', 'Track_id': '1pafeNyvbS0sVwzcsftYSZ', 'Prediction': 1}
{'Track Name': 'Midnight Crusade', 'Track_id': '4sykUbPIpLYRfZk1ljZcpj', 'Prediction': 1}
{'Track Name': 'Perdona

## The recommendations are:

In [95]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,Tears (feat. Louisa Johnson),7py16W5fWYLFFS6BElKAjn,1
1,Girls,5WSdMcWTKRdN1QYVJHJWxz,1
2,Race for the Prize - 2017 Remaster,1RZlGlz8eXT1KmJYpqFRzH,1
3,Work,32lmL4vQAAotg6MrJnhlQZ,1
4,One of Us,3zSaWBoojpQGDPO2oIL80H,1
5,Finally - Radio Edit,3YtL6y0eib6vgPmvF3mUed,1
6,Do I Wanna Know?,3jfr0TF6DQcOLat8gGn7E2,1
7,Unbreakable,15TC83LO1QAu0BZLRdHw01,1
8,Superheroes,186hvCTyrni4KT9nwIQ7zS,1
9,Wanna Sip,1pafeNyvbS0sVwzcsftYSZ,1
